In [27]:
import tweepy
import json
import twitter_credentials
user_file = 'twitter_user.json'

In [3]:
auth = tweepy.OAuthHandler(twitter_credentials.CONSUMER_KEY, twitter_credentials.CONSUMER_SECRET)
auth.set_access_token(twitter_credentials.ACCESS_TOKEN_KEY, twitter_credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [12]:
with open(user_file,'r') as uf:
    user_list = json.load(uf)

In [18]:
def get_tweets(userid):
    cursor = tweepy.Cursor(api.user_timeline,id = userid,tweet_mode = 'extended').items(200)
    tweet_list = []
    for i in cursor:
        tweet_list.append(i.full_text)
    return tweet_list

In [118]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import string
tk = TweetTokenizer(strip_handles = True)
tokenizer = RegexpTokenizer(r'\w+')
sia = SentimentIntensityAnalyzer()
stop_words = set(stopwords.words('english'))

In [147]:
def tweet_sentiment(tweet_list):
    overall_sentiment_score = 0
    for tweet in tweet_list:
        sentiment_score = sia.polarity_scores(tweet)
        overall_sentiment_score += sentiment_score['compound']
        
    if overall_sentiment_score == 0:
        overall_sentiment = 'neutral'
    elif overall_sentiment_score > 0:
        overall_sentiment = 'positive'
    else:
        overall_sentiment = 'negative'

    return overall_sentiment

In [144]:
def topten(tweet_list):
    full_text = []
    for tweet in tweet_list:
        tweet_tokens = tk.tokenize(tweet)
        filtered_sentence = []
        for w in tweet_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
        full_text += filtered_sentence
    full_textDist = nltk.FreqDist(full_text)
    return full_textDist.most_common(10)

In [148]:
for user in user_list:
    userid = user['user']
    tweet_list = get_tweets(userid)
    user['overall_sentiment'] = tweet_sentiment(tweet_list)
    user['most_common'] = topten(tweet_list)